## PEPFAR Linked View

In [1]:
import pandas as pd
import numpy as np
import folium
import plotly.graph_objects as go
from IPython.display import HTML
import json

def create_interactive_hiv_dashboard(map_data_path, chart_data_path):
    """
    Creates an interactive dashboard: clicking a country on the map
    shows only that country’s line in the chart.
    """
    # 1. Load and prepare map data
    df_map = pd.read_csv(map_data_path)
    df_map['country'] = (
        df_map['area']
        .str.replace('_', ' ', regex=False)
        .str.title()
    )
    grouped = (
        df_map
        .groupby(['country','latitude','longitude'])['data_value']
        .sum().reset_index()
    )
    grouped = grouped[
        grouped['latitude'].between(-90,90) &
        grouped['longitude'].between(-180,180)
    ]
    grouped['logv'] = np.log1p(grouped['data_value'])
    lo, hi = grouped['logv'].min(), grouped['logv'].max()
    # NEW – gentler sqrt scaling
    scale = (grouped['logv'] - lo) / (hi - lo)
    grouped['r'] = 3 + scale * 12


      # 2. Build the Folium map
    m = folium.Map(location=[0,0], zoom_start=2, tiles='CartoDB dark_matter')
    folium.GeoJson(
        'https://raw.githubusercontent.com/python-visualization/folium/master/examples/data/world-countries.json',
        style_function=lambda _: {'fillColor':'none','color':'#444','weight':0.5}
    ).add_to(m)

    countries = []
    for _, row in grouped.iterrows():
        countries.append(row['country'])
        folium.CircleMarker(
            location=[row['latitude'], row['longitude']],
            radius=row['r'],
            color='#ff3300',
            weight=0.5,
            fill=True,
            fill_color='#ff3300',
            fill_opacity=0.4,
            tooltip=f"{row['country']}: ${row['data_value']:,.0f}",
            className='hiv-marker'
        ).add_to(m)

    # 3. Inject JS into the map so each Leaflet circle posts a message on click
    m.save('hiv_map.html')
    #map_file = 'hiv_map.html'

    map_js = f"""
    <script>
      const countries = {json.dumps(countries)};
      // wait for Leaflet to finish drawing the markers
      setTimeout(() => {{
        document.querySelectorAll('.hiv-marker').forEach((el,i) => {{
          el.style.cursor = 'pointer';
          el.addEventListener('click', () => {{
            window.parent.postMessage(
              {{ type: 'countryClick', country: countries[i] }},
              '*'
            );
          }});
        }});
      }}, 500);
    </script>
    """
    m.get_root().html.add_child(folium.Element(map_js))
    # Build a custom legend overlay
    legend_html = '''
    <div style="
        position:absolute; bottom:30px; left:10px;
        background:rgba(0,0,0,0.7); color:white;
        padding:8px; border-radius:5px; font-size:12px;
        z-index:9999;">
      <strong>Expenditure scale</strong><br>
      <div style="display:flex;align-items:center;margin:4px 0;">
        <div style="width:5px;height:5px;background:#ff3300;border-radius:50%;margin-right:6px;"></div>$100K
      </div>
      <div style="display:flex;align-items:center;margin:4px 0;">
        <div style="width:9px;height:9px;background:#ff3300;border-radius:50%;margin-right:6px;"></div>$1M
      </div>
      <div style="display:flex;align-items:center;margin:4px 0;">
        <div style="width:15px;height:15px;background:#ff3300;border-radius:50%;margin-right:6px;"></div>$10M
      </div>
    </div>
    '''
    m.get_root().html.add_child(folium.Element(legend_html))

    map_html = m.get_root().render().replace('"', '&quot;')  # re‐save to include the JS


    # 4. Load and prepare chart data
    df_chart = pd.read_csv(chart_data_path)
    pepfar_countries = [
        "Botswana","Côte d'Ivoire","Ethiopia","Kenya","Mozambique",
        "Namibia","Nigeria","Rwanda","South Africa","Tanzania",
        "Uganda","Zambia","Guyana","Haiti"
    ]

    # build plotly figure and trace index
    fig = go.Figure()
    trace_index = {}
    idx = 0

    # non‐PEPFAR countries (hidden by default)
    all_ctrs = sorted(df_chart['Location'].unique())
    others = [c for c in all_ctrs if c not in pepfar_countries]
    for country in others:
        d = df_chart[df_chart['Location']==country].sort_values('Period')
        if d['FactValueNumericHigh'].nunique() > 1:
            fig.add_trace(go.Scatter(
                x=d['Period'],
                y=d['FactValueNumericHigh'],
                mode='lines+markers',
                name=country,
                line=dict(color='rgba(255, 51, 0,0.7)', width=3),
                marker=dict(size=6),
                visible=False
            ))
            trace_index[country] = idx
            idx += 1

    # PEPFAR countries
    for country in pepfar_countries:
        d = df_chart[df_chart['Location']==country].sort_values('Period')
        region_color = (
            'rgba(255, 51, 0,0.7)' if country in ["Guyana","Haiti"]
            else 'rgba(255, 51, 0,0.7)'
        )
        fig.add_trace(go.Scatter(
            x=d['Period'],
            y=d['FactValueNumericHigh'],
            mode='lines+markers',
            name=country,
            line=dict(color=region_color, width=3),
            marker=dict(size=6)
        ))
        trace_index[country] = idx
        idx += 1

    # total PEPFAR aggregate (legend only)
    agg = (
        df_chart[df_chart['Location'].isin(pepfar_countries)]
        .groupby('Period')['FactValueNumericHigh']
        .sum()
        .reset_index()
    )
    total_idx = idx
    fig.add_trace(go.Scatter(
        x=agg['Period'],
        y=agg['FactValueNumericHigh'],
        mode='lines+markers',
        name='Total PEPFAR',
        line=dict(color='white', width=3),
        marker=dict(size=8),
        visible=True
    ))

    # PEPFAR implementation line
    max_deaths = df_chart['FactValueNumericHigh'].max()
    fig.add_shape(dict(
        type='line',
        x0=2004, x1=2004,
        y0=0, y1=1,
        xref='x',
        yref='paper',               # ← use paper coordinates (0–1)
        line=dict(color='red', width=2, dash='dash')
    ))

    fig.add_annotation(dict(
        x=2004, y=1,
        xref='x', yref='paper',
        text='PEPFAR begins (2004)',
        showarrow=True,
        arrowhead=1,
        ax=0, ay=-30,
        font=dict(color='red', size=12)
    ))

    # Add a little legend text under the chart
    # fig.add_annotation(
    #     x=0.5, y=-0.15,
    #     xref='paper', yref='paper',
    #     text="<span style='color:red;'>●</span> Selected country   "
    #         "<span style='color:white;'>●</span> Total PEPFAR",
    #     showarrow=False,
    #     font=dict(size=12, color='white'),
    #     align='center'
    # )

        # — Dummy traces for legend only —
    # Red “country line” legend entry
    fig.add_trace(go.Scatter(
        x=[None], y=[None],
        mode='lines',
        line=dict(color='rgba(255,51,0,0.7)', width=3),
        name='Country line',
        showlegend=True
    ))
    # White “total” legend entry
    fig.add_trace(go.Scatter(
        x=[None], y=[None],
        mode='lines',
        line=dict(color='white', width=3),
        name='Total PEPFAR',
        showlegend=True
    ))




import pandas as pd
import numpy as np
import folium
import plotly.graph_objects as go
from IPython.display import HTML
import json

def create_interactive_hiv_dashboard(map_data_path, chart_data_path):
    """
    Creates an interactive dashboard: clicking a country on the map
    shows only that country's line in the chart.
    """
    # 1. Load and prepare map data
    df_map = pd.read_csv(map_data_path)
    df_map['country'] = (
        df_map['area']
        .str.replace('_', ' ', regex=False)
        .str.title()
    )
    grouped = (
        df_map
        .groupby(['country','latitude','longitude'])['data_value']
        .sum().reset_index()
    )
    grouped = grouped[
        grouped['latitude'].between(-90,90) &
        grouped['longitude'].between(-180,180)
    ]
    grouped['logv'] = np.log1p(grouped['data_value'])
    lo, hi = grouped['logv'].min(), grouped['logv'].max()
    # NEW – gentler sqrt scaling
    scale = (grouped['logv'] - lo) / (hi - lo)
    grouped['r'] = 3 + scale * 12


    # 2. Build the Folium map
    m = folium.Map(location=[0,0], zoom_start=2, tiles='CartoDB dark_matter')
    folium.GeoJson(
        'https://raw.githubusercontent.com/python-visualization/folium/master/examples/data/world-countries.json',
        style_function=lambda _: {'fillColor':'none','color':'#444','weight':0.5}
    ).add_to(m)

    countries = []
    for _, row in grouped.iterrows():
        countries.append(row['country'])
        folium.CircleMarker(
            location=[row['latitude'], row['longitude']],
            radius=row['r'],
            color='#ff3300',
            weight=0.5,
            fill=True,
            fill_color='#ff3300',
            fill_opacity=0.4,
            tooltip=f"{row['country']}: ${row['data_value']:,.0f}",
            className='hiv-marker'
        ).add_to(m)

    # 3. Inject JS into the map so each Leaflet circle posts a message on click
    m.save('hiv_map.html')
    #map_file = 'hiv_map.html'

    map_js = f"""
    <script>
      const countries = {json.dumps(countries)};
      // wait for Leaflet to finish drawing the markers
      setTimeout(() => {{
        document.querySelectorAll('.hiv-marker').forEach((el,i) => {{
          el.style.cursor = 'pointer';
          el.addEventListener('click', () => {{
            window.parent.postMessage(
              {{ type: 'countryClick', country: countries[i] }},
              '*'
            );
          }});
        }});
      }}, 500);
    </script>
    """
    m.get_root().html.add_child(folium.Element(map_js))
    # Build a custom legend overlay
    legend_html = '''
    <div style="
        position:absolute; bottom:30px; left:10px;
        background:rgba(0,0,0,0.7); color:white;
        padding:8px; border-radius:5px; font-size:12px;
        z-index:9999;">
      <strong>Expenditure scale</strong><br>
      <div style="display:flex;align-items:center;margin:4px 0;">
        <div style="width:5px;height:5px;background:#ff3300;border-radius:50%;margin-right:6px;"></div>$100K
      </div>
      <div style="display:flex;align-items:center;margin:4px 0;">
        <div style="width:9px;height:9px;background:#ff3300;border-radius:50%;margin-right:6px;"></div>$1M
      </div>
      <div style="display:flex;align-items:center;margin:4px 0;">
        <div style="width:15px;height:15px;background:#ff3300;border-radius:50%;margin-right:6px;"></div>$10M
      </div>
    </div>
    '''
    m.get_root().html.add_child(folium.Element(legend_html))

    map_html = m.get_root().render().replace('"', '&quot;')  # re‐save to include the JS


    # 4. Load and prepare chart data
    df_chart = pd.read_csv(chart_data_path)
    pepfar_countries = [
        "Botswana","Côte d'Ivoire","Ethiopia","Kenya","Mozambique",
        "Namibia","Nigeria","Rwanda","South Africa","Tanzania",
        "Uganda","Zambia","Guyana","Haiti"
    ]

    # build plotly figure and trace index
    fig = go.Figure()
    trace_index = {}
    idx = 0

    # non‐PEPFAR countries (hidden by default)
    all_ctrs = sorted(df_chart['Location'].unique())
    others = [c for c in all_ctrs if c not in pepfar_countries]
    for country in others:
        d = df_chart[df_chart['Location']==country].sort_values('Period')
        if d['FactValueNumericHigh'].nunique() > 1:
            fig.add_trace(go.Scatter(
                x=d['Period'],
                y=d['FactValueNumericHigh'],
                mode='lines+markers',
                name=country,
                line=dict(color='rgba(255, 51, 0,0.7)', width=3),
                marker=dict(size=6),
                visible=False
            ))
            trace_index[country] = idx
            idx += 1

    # PEPFAR countries
    for country in pepfar_countries:
        d = df_chart[df_chart['Location']==country].sort_values('Period')
        region_color = (
            'rgba(255, 51, 0,0.7)' if country in ["Guyana","Haiti"]
            else 'rgba(255, 51, 0,0.7)'
        )
        fig.add_trace(go.Scatter(
            x=d['Period'],
            y=d['FactValueNumericHigh'],
            mode='lines+markers',
            name=country,
            line=dict(color=region_color, width=3),
            marker=dict(size=6)
        ))
        trace_index[country] = idx
        idx += 1

    # total PEPFAR aggregate (legend only)
    agg = (
        df_chart[df_chart['Location'].isin(pepfar_countries)]
        .groupby('Period')['FactValueNumericHigh']
        .sum()
        .reset_index()
    )
    total_idx = idx
    fig.add_trace(go.Scatter(
        x=agg['Period'],
        y=agg['FactValueNumericHigh'],
        mode='lines+markers',
        name='Total PEPFAR',
        line=dict(color='white', width=3),
        marker=dict(size=8),
        visible=True
    ))

    # PEPFAR implementation line
    max_deaths = df_chart['FactValueNumericHigh'].max()
    fig.add_shape(dict(
        type='line',
        x0=2004, x1=2004,
        y0=0, y1=1,
        xref='x',
        yref='paper',               # ← use paper coordinates (0–1)
        line=dict(color='red', width=2, dash='dash')
    ))

    fig.add_annotation(dict(
        x=2004, y=1,
        xref='x', yref='paper',
        text='PEPFAR begins (2004)',
        showarrow=True,
        arrowhead=1,
        ax=0, ay=-30,
        font=dict(color='red', size=12)
    ))

    # Add a little legend text under the chart
    # fig.add_annotation(
    #     x=0.5, y=-0.15,
    #     xref='paper', yref='paper',
    #     text="<span style='color:red;'>●</span> Selected country   "
    #         "<span style='color:white;'>●</span> Total PEPFAR",
    #     showarrow=False,
    #     font=dict(size=12, color='white'),
    #     align='center'
    # )

    # — Dummy traces for legend only —
    # Red "country line" legend entry
    fig.add_trace(go.Scatter(
        x=[None], y=[None],
        mode='lines',
        line=dict(color='rgba(255,51,0,0.7)', width=3),
        name='Country line',
        showlegend=True
    ))
    # White "total" legend entry
    fig.add_trace(go.Scatter(
        x=[None], y=[None],
        mode='lines',
        line=dict(color='white', width=3),
        name='Total PEPFAR',
        showlegend=True
    ))


    # ADD AXIS LABELS HERE
    fig.update_layout(
        xaxis=dict(
            title=dict(
                text='Year',
                font=dict(size=14, color='white')
            ),
            titlefont=dict(size=14, color='white')
        ),
        yaxis=dict(
            title=dict(
                text='HIV-Related Deaths',
                font=dict(size=14, color='white')
            ),
            titlefont=dict(size=14, color='white')
        ),
        paper_bgcolor='black',
        plot_bgcolor='rgba(0,0,0,0.8)',
        font_color='white'
    )

    for i, trace in enumerate(fig.data):
      trace.visible = (i == total_idx)
    


    # Record index and serialize
    total_idx     = idx
    chart_json    = fig.to_json()
    mappings      = {'traceIndex': trace_index, 'totalIdx': total_idx}
    mappings_json = json.dumps(mappings)

    # Build the dashboard HTML

    dashboard_html = f"""
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8">
  <title>The Impact of PEPFAR</title>
  <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
  <style>
    html, body {{ margin:0; padding:0; height:100%; background:#000; color:#fff; overflow:hidden; }}
    .header {{ position:fixed; top:0; left:0; right:0; background:rgba(0,0,0,0.9);
               padding:12px; text-align:center; font-size:24px; z-index:1000; }}
    .subheader {{ text-align:center; font-size:14px; margin-top:4px; }}
    #dashboard-container {{ position:absolute; top:80px; bottom:0; left:0; right:0;
                            display:flex; }}
    #map-container, #chart-container {{ flex:1; display:flex; flex-direction:column; }}
    #map-container {{ border-right:1px solid #333; }}
    .panel-title {{ padding:8px; text-align:center; font-size:16px; background:rgba(0,0,0,0.7); }}
    iframe.map {{ flex:1; width:100%; height:100%; border:none; }}
    .selection {{ text-align:center; margin:8px 0; font-size:14px; }}
    .selection span {{ color:red; font-weight:bold; }}
    #reset-btn {{ margin:0 auto 12px; padding:6px 12px; background:#444; border:none;
                  border-radius:4px; color:#fff; cursor:pointer; }}
    #reset-btn:hover {{ background:#666; }}
    #plot {{ flex:1; width:100%; height:100%; }}
  </style>
</head>
<body>
  <div class="header">
    The Impact of PEPFAR
    <div class="subheader">Click a country bubble to show the trend of HIV-related deaths since 2000</div>
  </div>

  <div id="dashboard-container">
    <!-- Map panel -->
    <div id="map-container">
      <div class="panel-title">U.S. HIV Expenditure for Key Populations</div>
      <iframe
        class="map"
        sandbox="allow-scripts allow-same-origin"
        srcdoc="{map_html}"
      ></iframe>
    </div>

    <!-- Chart panel -->
    <div id="chart-container">
      <div class="panel-title">HIV-Related Deaths (2000–2023)</div>
      <div class="selection">
        Currently selected: <span id="current-country">Total PEPFAR</span>
      </div>
      <button id="reset-btn">Show Total PEPFAR</button>
      <div id="plot"></div>
    </div>
  </div>

  <script>
    const cfg = {mappings_json};
    const chartData = {chart_json};

    // draw initial chart
    Plotly.newPlot('plot', chartData.data, chartData.layout);
    window._baseData   = chartData.data;
    window._baseLayout = chartData.layout;

    function highlightCountry(country) {{
      document.getElementById('current-country').textContent = country;
      let idx = (country==='Total PEPFAR' ? cfg.totalIdx : cfg.traceIndex[country]);
      if (idx===undefined) {{
        Plotly.react('plot',
          window._baseData.map(t=>({{...t,visible:false}})),
          window._baseLayout
        );
      }} else {{
        Plotly.react('plot',
          window._baseData.map((t,i)=>({{...t,visible:i===idx}})),
          window._baseLayout
        );
      }}
    }}

    window.addEventListener('message', e => {{
      if (e.data?.type==='countryClick') highlightCountry(e.data.country);
    }});

    document.getElementById('reset-btn').onclick = () => {{
      highlightCountry('Total PEPFAR');
    }};
  </script>
</body>
</html>
"""

    with open('dashboard.html','w') as f:
      f.write(dashboard_html)   




In [2]:
dashboard = create_interactive_hiv_dashboard(
    map_data_path="../../data/clean_data/hiv_expenditure_lat_long.csv",
    chart_data_path="../../data/raw_data/WHO_HIV_deaths.csv"
)
display(dashboard)

None